# PART 1

In [123]:
import pandas as pd
import numpy as np

In [124]:
#read the table after converting to csv
postal_code = pd.read_csv('Postalcode.csv')
postal_code.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [125]:
#drop rows if borough is no assigned 
postal_code = postal_code[postal_code.Borough != 'Not assigned']
postal_code.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [126]:
#group multiple neighbourhoods
postal_code['Neighbourhood'] = postal_code.groupby(['Postcode'])['Neighbourhood'].transform(lambda x: ','.join(x))
postal_code.drop_duplicates(inplace = True)
postal_code.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge,Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens,Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson,Garden District"


In [127]:
#set unassigned neighbourhood equals to borough
postal_code.loc[postal_code['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = postal_code['Borough']

In [128]:
postal_code.shape

(103, 3)

# PART 2

In [129]:
#Read coordinates csv into dataframe
coordinates = pd.read_csv('Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [130]:
#join two dataframes on Postal Code
postal_code.rename(columns = {'Postcode': 'Postal Code'}, inplace = True)
df = pd.merge(postal_code, coordinates, on='Postal Code', how='left')
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


# Part 3

## I love sushi, and I can't live without it. However, I need to workout if I eat too much. So I'm goona focus on the location of sushi restaurants and gyms to figure out the best place to live in Toronto. 

In [131]:
from geopy.geocoders import Nominatim
import folium
import requests
import json

In [132]:
#Visualize the location of different boroughs in the map.
#get the location of Toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [133]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [134]:
#foursquare credentials
CLIENT_ID = 'SLOTL4BOPTRC0G2P4OH13AXOIZLVZFNFP3EOD14CYQ32R3FW' # your Foursquare ID
CLIENT_SECRET = 'EOMYXZBVSGBIGO32XKWCHRNCWA2SZ40HYLK50J1E4XTZGJVR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SLOTL4BOPTRC0G2P4OH13AXOIZLVZFNFP3EOD14CYQ32R3FW
CLIENT_SECRET:EOMYXZBVSGBIGO32XKWCHRNCWA2SZ40HYLK50J1E4XTZGJVR


In [135]:
#define a function to get the number of sushi restaurants and gyms around a neighborhood(2000 meters)

In [143]:
def get_number(query,latitude,longitude):
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius=2000&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    query)
    
    results = requests.get(url).json()
    #num = len(results['response']['venues'])
    return results

In [145]:
#append the information to dataframe
NumSushi = []
NumGym = []

for i in range(0,len(df)):
    num = get_number('gym',df.loc[i]['Latitude'],df.loc[i]['Longitude'])
    NumGym.append(num)
    num = get_number('sushi',df.loc[i]['Latitude'],df.loc[i]['Longitude'])
    numSushi.append(num)
df['NumGym']=NumGym
df['NumSushi']=NumSushi

NameError: name 'numSushi' is not defined

In [ ]:
#using matplotlib to get a sense of K
import matplotlib.pyplot as plt
%matplotlib inline

df.plot(kind='scatter',x='NumSushi',y='NumGym')

In [ ]:
#use K = 4
from sklearn.cluster import KMeans
cols = ['NumSushi','NumGym']
df_cluster = df[cols]
kmeans = KMeans(n_clusters=4, random_state=0).fit(df_cluster)
kmeans.labels_

In [ ]:
df.insert(0,'labels',kmeans.labels_)
df.head()

In [122]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighbourhood'], df['labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Conclusion: We can see most gyms and sushi restaurants are in the downtown area, while the other clusters appears to be in the outer range of the city. but the rent in downtown is expensive. Surprisingly, there is a neighborhood where the label is '1'(Willowdale South),same as downtown area, but locates in the outer town. So it is going to be best choice.